In [7]:
import pandas as pd
from pymongo import MongoClient
import csv
from bs4 import BeautifulSoup
from time import sleep
import re
import requests
import html5lib
from IPython.display import clear_output
from datetime import datetime
import re

In [8]:
client = MongoClient("mongodb://localhost:27017/")
db = client["UTA_Enrollment"]
articles = db["articles"]

In [9]:
def solve(s):                                             
    return re.sub(r'(\d)(st|nd|rd|th)', r'\1', s)

In [12]:
def remove_redun_rows(df, default_cols, cont_col_subset = ["Category", "Headline", "Summary", "Entire_News", "News_Link"], meta_col_subset = ["Datetime"]):
    """It removes faulty or duplicate rows
    If there are more columns in the given dataframe than default, this removes those rows that have such more columns. 
    If there are less columns in the given dataframe than default, it returns "None", thereby marking them unusable. 
    If there are duplicated content in any "cont_col_subset", it drops the extra rows except the latest entry.
    If there are rows with missing values for important columns, it removes such rows. 
    If there are rows with missing values for non-important columns, it ignores them, but informs there existence. 
    It returns the trimmed dataframe as the output in all cases except when the number of columns are less than default,
    and prints any missing values in non-important columns"""
    skip = False
    if list(df.columns) != list(default_cols):
        if len(df.columns) == len(default_cols):
            print("There seems to be some error in columns names")
        elif len(df.columns) < len(default_cols):
            print("The given DataFrame seems to have some missing columns")
            df = None # Marking the df useless
            skip = True # Skipping the DataFrame operations
        elif len(df.columns) > len(default_cols):
            print("The given DataFrame seems to have more columns than required")
            df_xtra_col_idx = df.loc[:, df.columns[len(default_cols):]].isnull().any(axis = 1)
            df = df.loc[df.index[df_xtra_col_idx], default_cols]
    if not skip:
        df = df.drop_duplicates(subset = cont_col_subset)
        df = df.dropna(subset = cont_col_subset+meta_col_subset)
    return df

In [18]:
#looking at the website of indian express,they have a task bar on the first page for broad categories of news
#following those broad categories there were some obvious sub-categories where news were covered, these are
#mentioned here

categories = {
    'india':[''],
    'cities':[''],
    'sports':[
         'badminton', 'cricket', 'football', 
        'fifa', 'hockey', 'motor-sport', 'tennis', 
        'wwe-wrestling'
        ], 
    'business':[
        'aviation', 'banking-and-finance',
        'companies', 'economy', 'market'
        ],
    'technology':[
        'gadgets','laptops',
        'tech-reviews','science','techook'
        ],
    'entertainment':['bollywood','hollywood',
        'television','tamil','telugu','malayalam',
        'web-series','movie-review','box-office-collection'
        ]
}


csv_file = 'IndianExpress.csv'

ndf1 = pd.DataFrame(columns = [
    'Datetime',
    'Category',
    'Subcategory',
    'Headline',
    'Summary',
    'Entire_News',
    'Author',
    'News_Link'
])


ndf1.index.name = "Index"


#suppose a webpage doesn't exisit for a particular category->sub-category then this data frame will be used to 
#append that error! page not found type of error

edf1 = pd.DataFrame(columns = [
    'Website Link',
    'Error'
])


if __name__ == '__main__':
    news_website_link = "https://indianexpress.com"
    
    for category in categories:
        for sub_category in categories[category]:
                
            for page in range(0,1):  #number of pages to search for
                #link for a particular news webpage
                news_url = news_website_link + "/section/" + category + "/" + sub_category + "/" + "page/" + str(page) + "/"
                
                #scrap the html version of the webpage
                sleep(0.05)

                try:
                    news_html_page = requests.get(news_url)

                except Exception as e:
                        
                        # Pushing the error website link data to the DataFrame
                        edf1 = edf1.append({
                            'Website Link' : news_urll, 
                            'Error' : e
                        },
                        ignore_index = True)
                        continue
                
                #interpret the html file to actual sequence of words
                news_html_interpreted = BeautifulSoup(news_html_page.content,'html.parser')
                
                # Fetching news url for every news on a page from headings and titles
                news_head_list = news_html_interpreted.findAll('h2')

                print(category)
                print(sub_category)
                print(page) 
                
                i = 0
                for news_head in news_head_list:

                    try:
                        news_urll = news_head.find('a')['href']

                        if articles.find_one({"News_Link": news_urll}):
                            break    

                        news_html_data = requests.get(news_urll)

                        news_html_interpretation = BeautifulSoup(news_html_data.content, 'html.parser')

                         # Fetching the headline and brief description of the news
                        title = news_html_interpretation.find('h1',attrs={'class':'native_story_title'}).get_text()
                        summary = news_html_interpretation.find('h2', attrs={'class':'synopsis'}).get_text()

                        # Fetching the author, date and time of publish
                        publishing_details = news_html_interpretation.find('div', attrs={'class':'editor'})
                        publisher = publishing_details.find('a').get_text()
                        date_time = publishing_details.find('span').get_text()

                    #     dt = date_time.split()
                    #     if "Updated:" in dt:
                    #         dt.remove("Updated:")
                    #     # dt.remove("Updated:")
                    #     date_time = dt[0] + " " + dt[1] + " " + dt[2] + " " + dt[3] + " " + dt[4]
                    #     datetime_object = datetime.strptime(solve(date_time), '%B %d, %Y %I:%M:%S %p')
                        
                        
                    #     # Fetching the entire content of the news 
                    #     report = ""
                    #     report_paras = news_html_interpretation.findAll('p')
                    #     for para in range(len(report_paras)):
                    #         report += report_paras[para].get_text()

                    #     print(i, end = " ")
                    #     i = i + 1
                        
                    # except Exception as e:
                    #     edf1 = edf1.append({
                    #         'Website Link' : news_urll, 
                    #         'Error' : e
                    #     },
                    #     ignore_index = True)
                    #     continue
                    
                    # # Pushing the scrapped news data to the DataFrame
                    # ndf1 = ndf1.append({
                    #     'Datetime' : datetime_object,
                    #     # 'Time' : time,
                    #     'Category' : category.capitalize(),
                    #     'Subcategory' : sub_category.capitalize(),
                    #     'Headline' : title,
                    #     'Summary' : summary, 
                    #     'Entire_News' : report,
                        # 'Author' : publisher.capitalize(), 
                        # 'News_Link' : news_urll
                        # },
                        # ignore_index = True)
                        # Handle date formatting
                        dt = date_time.split()

                        # Remove "Updated:" only if it exists
                        if "Updated:" in dt:
                            dt.remove("Updated:")

                        date_time = " ".join(dt[:5])  # Join the first 5 elements back into a string
                        datetime_object = datetime.strptime(solve(date_time), '%B %d, %Y %I:%M:%S %p')

                        # Create a new DataFrame for the scraped data
                        new_data = pd.DataFrame([{
                            'Datetime': datetime_object,
                            'Category': category.capitalize(),
                            'Subcategory': sub_category.capitalize(),
                            'Headline': title,
                            'Summary': summary,
                            'Entire_News': report,
                            'Author': publisher.capitalize(),
                            'News_Link': news_urll
                        }])

                        # Concatenate the new data to the existing DataFrame
                        ndf1 = pd.concat([ndf1, new_data], ignore_index=True)

                        # If there are errors, create a DataFrame and concatenate
                        error_data = pd.DataFrame([{
                            'Website Link': news_urll,
                            'Error': e
                        }])

                        edf1 = pd.concat([edf1, error_data], ignore_index=True)


    # if not ndf1.empty:

    #     temp = remove_redun_rows(ndf1, default_cols = ndf1.columns)
    #     temp = cat_reformat(temp)

    #     # Storing the DataFrame to a .csv file
    #     for i in articles.find().sort("ID",-1).limit(1):
    #         last_id = int(i["ID"]) + 1
    #         temp.index = temp.index + last_id
    #     temp.insert(loc=0, column='ID', value=temp.index)
    #     data_dict = temp.to_dict("records")
    #     articles.insert_many(data_dict)

    #     # Storing the unscrapped(error) data to csv
        # edf1.to_csv('Error_Data_RW.csv', index = False, encoding='utf')


SyntaxError: unexpected EOF while parsing (1172823309.py, line 198)

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep
import logging
from typing import Dict, List, Optional
from urllib.parse import urljoin
import os
import csv
from pathlib import Path

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='scraper.log'
)

class CSVHandler:
    def __init__(self, filename: str):
        self.filename = filename
        self.file_exists = Path(filename).exists()
        
    def save_to_csv(self, data: pd.DataFrame, mode: str = 'a') -> bool:
        try:
            # If file doesn't exist, write with header
            if not self.file_exists or mode == 'w':
                data.to_csv(self.filename, mode='w', index=True, encoding='utf-8')
                self.file_exists = True
            else:
                # Append without header
                data.to_csv(self.filename, mode='a', header=False, index=True, encoding='utf-8')
            return True
        except Exception as e:
            logging.error(f"Error saving to CSV {self.filename}: {str(e)}")
            return False
            
    def backup_existing_file(self) -> None:
        if self.file_exists:
            backup_name = f"{self.filename[:-4]}_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            try:
                os.rename(self.filename, backup_name)
                logging.info(f"Created backup: {backup_name}")
            except Exception as e:
                logging.error(f"Error creating backup: {str(e)}")

class NewsArticle:
    def __init__(self, url: str, category: str, subcategory: str):
        self.url = url
        self.category = category
        self.subcategory = subcategory
        self.html = None
        self.soup = None
        
    def fetch_content(self) -> bool:
        try:
            response = requests.get(self.url, timeout=10)
            response.raise_for_status()
            self.html = response.content
            self.soup = BeautifulSoup(self.html, 'html.parser')
            return True
        except Exception as e:
            logging.error(f"Error fetching {self.url}: {str(e)}")
            return False
            
    def parse_article(self) -> Optional[Dict]:
        if not self.soup:
            return None
            
        try:
            # Extract article components
            title = self.soup.find('h1', attrs={'class': 'native_story_title'})
            summary = self.soup.find('h2', attrs={'class': 'synopsis'})
            publishing_details = self.soup.find('div', attrs={'class': 'editor'})
            
            if not all([title, summary, publishing_details]):
                return None
                
            publisher = publishing_details.find('a')
            date_time = publishing_details.find('span')
            
            if not all([publisher, date_time]):
                return None
                
            # Parse datetime
            dt_text = date_time.get_text().split()
            if "Updated:" in dt_text:
                dt_text.remove("Updated:")
            clean_dt = " ".join(dt_text[:5])
            datetime_obj = datetime.strptime(clean_dt, '%B %d, %Y %I:%M:%S %p')
            
            # Get full article text
            report_paras = self.soup.findAll('p')
            report = " ".join(para.get_text() for para in report_paras)
            
            return {
                'Datetime': datetime_obj,
                'Category': self.category.capitalize(),
                'Subcategory': self.subcategory.capitalize(),
                'Headline': title.get_text().strip(),
                'Summary': summary.get_text().strip(),
                'Entire_News': report.strip(),
                'Author': publisher.get_text().strip().capitalize(),
                'News_Link': self.url
            }
            
        except Exception as e:
            logging.error(f"Error parsing {self.url}: {str(e)}")
            return None

class IndianExpressScraper:
    def __init__(self, base_url: str, categories: Dict[str, List[str]], batch_size: int = 10):
        self.base_url = base_url
        self.categories = categories
        self.batch_size = batch_size
        self.articles_df = pd.DataFrame(columns=[
            'Datetime', 'Category', 'Subcategory', 'Headline',
            'Summary', 'Entire_News', 'Author', 'News_Link'
        ])
        self.errors_df = pd.DataFrame(columns=['Website Link', 'Error'])
        self.articles_csv = CSVHandler('IndianExpress.csv')
        self.errors_csv = CSVHandler('Error_Data_RW.csv')
        
    def build_url(self, category: str, subcategory: str, page: int) -> str:
        return urljoin(
            self.base_url,
            f"/section/{category}/{subcategory}/page/{page}/"
        )
        
    def save_batch(self) -> None:
        """Save current batch of articles and clear DataFrame"""
        if not self.articles_df.empty:
            self.articles_csv.save_to_csv(self.articles_df)
            self.articles_df = pd.DataFrame(columns=self.articles_df.columns)
            
    def scrape_page(self, url: str, category: str, subcategory: str) -> None:
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all news headlines
            for news_head in soup.findAll('h2'):
                link = news_head.find('a')
                if not link or not link.get('href'):
                    continue
                    
                article_url = link['href']
                article = NewsArticle(article_url, category, subcategory)
                
                if article.fetch_content():
                    article_data = article.parse_article()
                    if article_data:
                        self.articles_df = pd.concat([
                            self.articles_df,
                            pd.DataFrame([article_data])
                        ], ignore_index=True)
                        logging.info(f"Successfully scraped: {article_url}")
                        
                        # Save batch if we've reached batch_size
                        if len(self.articles_df) >= self.batch_size:
                            self.save_batch()
                            
                # Respect the website by adding delay
                sleep(0.1)
                
        except Exception as e:
            error_data = pd.DataFrame([{'Website Link': url, 'Error': str(e)}])
            self.errors_csv.save_to_csv(error_data)
            logging.error(f"Error scraping page {url}: {str(e)}")
            
    def run(self, pages_per_category: int = 1) -> None:
        # Backup existing files
        self.articles_csv.backup_existing_file()
        self.errors_csv.backup_existing_file()
        
        for category, subcategories in self.categories.items():
            for subcategory in subcategories:
                if not subcategory:  # Skip empty subcategories
                    continue
                    
                for page in range(pages_per_category):
                    url = self.build_url(category, subcategory, page)
                    logging.info(f"Scraping {category}/{subcategory} - page {page}")
                    self.scrape_page(url, category, subcategory)
                    
        # Save any remaining articles
        self.save_batch()

if __name__ == '__main__':
    # Categories dictionary
    categories = {
        'india': [''],
        'cities': [''],
        'sports': [
            'badminton', 'cricket', 'football',
            'fifa', 'hockey', 'motor-sport', 'tennis',
            'wwe-wrestling'
        ],
        'business': [
            'aviation', 'banking-and-finance',
            'companies', 'economy', 'market'
        ],
        'technology': [
            'gadgets', 'laptops',
            'tech-reviews', 'science', 'techook'
        ],
        'entertainment': [
            'bollywood', 'hollywood',
            'television', 'tamil', 'telugu', 'malayalam',
            'web-series', 'movie-review', 'box-office-collection'
        ]
    }
    
    # Initialize and run scraper
    scraper = IndianExpressScraper(
        base_url="https://indianexpress.com",
        categories=categories,
        batch_size=10  # Save to CSV after every 10 articles
    )
    
    scraper.run(pages_per_category=1)